# Model VAR (package [`sktime`](https://www.sktime.net/en/stable/index.html))

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.var import VAR
from sktime.forecasting.varmax import VARMAX

from sktime.utils.plotting import plot_series

from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError # Метрики MSE, MAE
from sktime.forecasting.model_evaluation import evaluate

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

In [ ]:
rates = web.DataReader(name=['TB3MS', 'GS10', 'AAA', 'BAA'], data_source='fred', start='2000-01-01', end='2023-12-31')
rates.index = pd.period_range(start='2000-01', end='2023-12', freq='M')

y = rates.diff().dropna()

y.plot(subplots=True)
plt.show()

## Fitting VAR of given order

In [ ]:
forecaster = VAR(maxlags=2, ic=None, trend='c')

forecaster.fit(y)

forecaster.get_fitted_params()

## Fitting VAR of "optimal" order

In [ ]:
forecaster = VAR(maxlags=6, ic='bic', trend='c')

forecaster.fit(y)

forecaster.get_fitted_params()

## Fitting model of given order (VARMAX)

In [ ]:
forecaster = VARMAX(order=(2,0), trend='c')

forecaster.fit(y)

# Forecasting

In [ ]:
forecaster = VARMAX(order=(2,0), trend='c')

y_pred = forecaster.fit_predict(y, fh=np.arange(1,11))
y_pred

In [ ]:
fig, axs = plt.subplots(nrows=y.shape[1], ncols=1)

for i in range(y.shape[1]):
	plot_series(y.iloc[:,i].tail(50), y_pred.iloc[:,i], ax=axs[i], markers=['.', '.'])

plt.show()

## Cross-validation

In [ ]:
# model specification
forecaster = VAR(maxlags=2, ic=None, trend='c')

# train-test split (first 100 go to train) 
cv = SlidingWindowSplitter(fh=np.arange(1, 11), initial_window=100, step_length=10)

# initializing metrics
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError()]

df = evaluate(forecaster=forecaster, y=y, cv=cv, strategy="refit", return_data=False, scoring=metric)

# averaged MSE & MAE
df.iloc[:,[0,1]].mean()